In [56]:
import os
import sys
import pandas as pd 

sys.path.append("../python")

import matplotlib.pyplot as plt
import numpy as np
from gurobipy import *
import math

In [94]:
X = pd.read_csv('../heuristic/data/X.csv')
Y = pd.read_csv('../heuristic/data/Y.csv')
Z = pd.read_csv('../heuristic/data/Z.csv')

X = X.drop(columns = X.columns[0])
Y = Y.drop(columns = Y.columns[0])

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [89]:
cluster_X_0_test = X.iloc[:50,0:2].to_numpy()
cluster_Y_0_test = Y.iloc[:50,0:2].to_numpy()

In [97]:
sous_tableau_1, sous_tableau_2, sous_tableau_3 = np.split(X.to_numpy(), 3)


In [101]:
len(sous_tableau_3)

13334

In [58]:
cluster_X_0 = X.iloc[:1334].to_numpy()
cluster_X_1 = X.iloc[1334:2668].to_numpy()
cluster_X_2 = X.iloc[2668:4003].to_numpy()

cluster_Y_0 = Y.iloc[:1334].to_numpy()
cluster_Y_1 = Y.iloc[1334:2668].to_numpy()
cluster_Y_2 = Y.iloc[2668:4003].to_numpy()

In [95]:
def opti_cluster(cluster_X, cluster_Y): 
        # Instanciation du modèle
    m = Model('cluster')

    #Nombre de features
    N_CRITERIA = len(cluster_X[0])
    #Nombre de segment/breakpoints dans toutes les fonctions de score, possible de l'adapter pour chaque feature 
    L = 5
    #Nombre d'occurence dans le dataset/produits traités 
    N_COMPARAISON = len(cluster_X)
    #Majorant contrainte 
    M=4
    #definition de epsilon
    epsilon = 0.001

    min_int_X = cluster_X.min(axis=0) 
    min_int_Y = cluster_Y.min(axis=0)
    mins = np.min(np.vstack((min_int_X, min_int_Y)), axis = 0)

    max_int_X = cluster_X.max(axis=0)
    max_int_Y = cluster_Y.max(axis=0)
    maxs = np.max(np.vstack((max_int_X, max_int_Y)), axis = 0)

    #Variables
    criteria = [[(m.addVar(name=f"u_{j}{l}")) for l in range(L+1)]for j in range(N_CRITERIA)]
    sigma_Xplus = [(m.addVar(name=f"x_sigma+{i}")) for i in range(N_COMPARAISON)]
    sigma_Xminus = [(m.addVar(name=f"x_sigma-{i}")) for i in range(N_COMPARAISON)]
    sigma_Yplus = [(m.addVar(name=f"y_sigma+{i}")) for i in range(N_COMPARAISON)]
    sigma_Yminus = [(m.addVar(name=f"y_sigma-{i}")) for i in range(N_COMPARAISON)]

   
    m.update()


    def segment(i, j, X):
        x = X[i][j]
        return math.floor(L * (x - mins[j]) / (maxs[j] - mins[j]))

    def x_seg(j, seg):
        return mins[j] + seg * (maxs[j] - mins[j]) / L

    def u(i,j,X,eval : bool = False):
        get_val = (lambda v: v.X) if eval else (lambda v: v)
        x = X[i][j]
        
        if x == maxs[j]:
            return get_val(criteria[j][-1]) 
        else:
            segm = segment(i, j, X)
            x_segm = x_seg(j, segm)
            x_segm_1 = x_seg(j, segm +1)    
            slope = (x - x_segm)/(x_segm_1 - x_segm)
            width = criteria[j][segm+1] - criteria[j][segm]
            S =  slope*width + criteria[j][segm]
        return S


    #définissons la somme des fonctions de score pour un produit (de rang i)
    def s(i, X, eval : bool = False):
        if not eval : 
            return quicksum(u(i,j,X,False) for j in range(N_CRITERIA))
        else : 
            return sum(u(i,j,X,False) for j in range(N_CRITERIA))

    def s_total(i, X): 
        return s(i, X, False) + sigma_Xminus[i] - sigma_Xminus[i] 

    # Ajout des contraintes
                
    # Constraint for sigma_Xminus
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xminus[i] >= 0)

    # Constraint for sigma_Xplus
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xplus[i] >= 0)
                        
    # Constraint for sigma_Yminus
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yminus[i] >= 0)

    # Constraint for sigma_Yplus
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yplus[i] >= 0)

    # contrainte 3
    for j in range(N_CRITERIA):
        for l in range(L-1):
            m.addConstr(criteria[j][l+1] - criteria[j][l] >= epsilon)

    # contrainte 4
    m.addConstr(sum(criteria[j][L] for j in range(N_CRITERIA)) ==1)

    # contrainte 5
    for j in range(N_CRITERIA): 
        m.addConstr(criteria[j][0] == 0)    


    for i in range(N_COMPARAISON):
        m.addConstr(s_total(i, cluster_X) - s_total(i, cluster_Y) >= epsilon)

    m.update()

    # Flatten the lists of lists into a single list
    # sum_to_minize= 0 
    # for i in range(N_COMPARAISON): 
    #     sum_to_minize += sum(sigma_Xplus[i] + sigma_Xminus[i] + sigma_Yminus[i] + sigma_Yplus[i])
    

    sum_to_minize = sum(sigma for sigma in sigma_Xplus) + sum(sigma for sigma in sigma_Xminus) + sum(sigma for sigma in sigma_Yplus) + sum(sigma for sigma in sigma_Yminus)
    fonction_objectif = sum_to_minize
    #fonction_objectif = sum(sigma_Xplus[i] + sigma_Xminus[i] + sigma_Yminus[i] + sigma_Yplus[i]  for i in range(N_COMPARAISON))

    # Objective function
    m.setObjective(fonction_objectif, GRB.MINIMIZE)


    # Fonction Objectif
    #m.setObjective(sum(sigma_Xplus) + sum(sigma_Xminus) + sum(sigma_Yminus) + sum(sigma_Yplus), GRB.MINIMIZE)         
    # Paramétrage (mode mute)
    # m.Params.Presolve = 0
    # Résolution du PL
    m.optimize()
    #print("La solution optimale est (v0, v1) = {} avec pour objectif z = {}".format((v0.x, v1.x), m.objVal))
    for k in range(K):
        for j in range(N_CRITERIA):
            for l in range(L):
                print(criteria[j][l])



In [60]:
cluster = cluster_0
mins = cluster.min(axis=0)
maxs = cluster.max(axis=0)

In [61]:
cluster_X_0[0]
s(cluster_X_0[0])


array([0.13155 , 0.102793, 0.266773, 0.273273, 0.482818, 0.598529,
       0.443253, 0.079392, 0.696425, 0.205184])

In [103]:
opti_cluster(cluster_X_0_test,cluster_Y_0_test)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.1.0 23B74)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 261 rows, 212 columns and 537 nonzeros
Model fingerprint: 0x5de0efb5
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 1e+00]
Presolve removed 205 rows and 203 columns
Presolve time: 0.12s
Presolved: 56 rows, 9 columns, 289 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.061832e+00   0.000000e+00      0s

Solved in 18 iterations and 0.13 seconds (0.00 work units)
Infeasible model
<gurobi.Var u_00>
<gurobi.Var u_01>
<gurobi.Var u_02>
<gurobi.Var u_03>
<gurobi.Var u_04>
<gurobi.Var u_10>
<gurobi.Var u_11>
<gurobi.Var u_12>
<gurobi.Var u_13>
<gurobi.Var u_14>
<gurobi.Var u_00>
<gurobi.Var u_01>
<gur

In [80]:
L = [1,1,1,1]

sum(L[i] + L[i] +L[i] +L[i]  for i in range(len(L)))

16